In [1]:
import scipy.io as sio
import tensorflow as tf
import numpy as np
from google.colab import data_table
import pandas as pd
import string

### 下載資料並進行整理

In [2]:
!wget https://github.com/toonnyy8/mhmc/raw/master/HW_topic_detection/data.zip
!unzip data.zip

--2020-09-03 14:54:13--  https://github.com/toonnyy8/mhmc/raw/master/HW_topic_detection/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/toonnyy8/mhmc/master/HW_topic_detection/data.zip [following]
--2020-09-03 14:54:14--  https://raw.githubusercontent.com/toonnyy8/mhmc/master/HW_topic_detection/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2135310 (2.0M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.04M  9.25MB/s    in 0.2s    

2020-09-03 14:54:16 (9.25 MB/s) - ‘data.zip’ saved [2135310/2135310]

Archive:  data.zip
   creating: data/


In [26]:
training_data_path = './data/train/dialogues_train.txt'
training_answer_path = './data/train/dialogues_train_topic.txt'

testing_data_path = './data/test/dialogues_test.txt'
testing_answer_path = './data/test/dialogues_test_topic.txt'

def statis(cm):
  accu = 0
  for n in range(len(cm)):
    accu += cm[n][n]
  accu /= sum(sum(cm))
  return accu

class_weights = np.zeros(10)
token_dict = {"__padding__":0, "__cls__":1, "__mask__":3}

In [27]:
train_txts = open(training_data_path,'r').readlines()
train_sentence = []
train_max_len = 0
for train_txt in train_txts:
  sentence = train_txt.replace('\n','')
  sentence_list = sentence.split(' ')
  sentence_list = [ch for ch in sentence_list if ch != '']
  train_sentence.append(sentence_list)
  if len(sentence_list)>train_max_len:
    train_max_len = len(sentence_list)

In [28]:
test_txts = open(testing_data_path,'r').readlines()
test_sentence = []
test_max_len = 0
for test_txt in test_txts:
  sentence = test_txt.replace('\n','')
  sentence_list = sentence.split(' ')
  sentence_list = [ch for ch in sentence_list if ch != '']
  test_sentence.append(sentence_list)
  if len(sentence_list)>test_max_len:
    test_max_len = len(sentence_list)

In [64]:
# train_tokens = np.concatenate([np.ones((len(train_sentence),1),dtype="int32"),np.zeros((len(train_sentence),train_max_len),dtype="int32")],axis=1)
train_tokens = np.zeros((len(train_sentence),train_max_len),dtype="int32")

for line, sentence_list in enumerate(train_sentence):
  for idx, token in enumerate(sentence_list):
    if token_dict.get(token) == None:
      token_dict[token] = len(token_dict)
    train_tokens[line][idx+(train_max_len-len(sentence_list))] = token_dict[token]

In [65]:
train_tokens

array([[  0,   0,   0, ...,   4,  26,  14],
       [  0,   0,   0, ..., 128,  26,  14],
       [  0,   0,   0, ..., 147,  26,  14],
       ...,
       [  0,   0,   0, ..., 382,  26,  14],
       [  0,   0,   0, ..., 622,  26,  14],
       [  0,   0,   0, ...,  29,  26,  14]], dtype=int32)

In [67]:
# test_tokens = np.concatenate([np.ones((len(test_sentence),1),dtype="int32"),np.zeros((len(test_sentence),test_max_len),dtype="int32")],axis=1)
test_tokens = np.zeros((len(test_sentence),test_max_len),dtype="int32")

for line, sentence_list in enumerate(test_sentence):
  for idx, token in enumerate(sentence_list):
    if token_dict.get(token) == None:
      test_tokens[line][idx+(test_max_len-len(sentence_list))] = token_dict["__mask__"]
    else:
      test_tokens[line][idx+(test_max_len-len(sentence_list))] = token_dict[token]

In [51]:
train_answers = open(training_answer_path,'r').readlines()
train_ans = []
for train_answer in train_answers:
  train_ans.append(int(train_answer))
  class_weights[int(train_answer)-1] += 1

class_weights = (np.sum(class_weights)/class_weights)/10

In [39]:
test_answers = open(testing_answer_path,'r').readlines()
test_ans = []
for test_answer in test_answers:
  test_ans.append(int(test_answer))

In [80]:
def random_masked(inp,rate,mask_id):
  return tf.where(
    tf.greater_equal(tf.random.uniform(inp.shape,0,1),rate),
    inp,
    mask_id,
  )

# random_masked(train_tokens,0.1,token_dict["__mask__"]) 

In [79]:
def add_cls(inp,cls_id):
  return tf.concat([tf.ones((inp.shape[0],1),"int32"),inp],axis=1)
# add_cls(
#   random_masked(
#     train_tokens,
#     0.1,
#     token_dict["__mask__"],
#   ),
#   token_dict["__cls__"]
# )

In [83]:
# tf.keras.layers.Embedding(len(token_dict), 64)(
#   add_cls(
#     random_masked(
#       train_tokens,
#       0.1,
#       token_dict["__mask__"],
#     ),
#     token_dict["__cls__"]
#   )   
# ).shape

TensorShape([10494, 869, 64])